# How to use with Azure OpenAI Service

PSOpenAI has limited supports for Azure OpenAI Service.

## Setup

For the following sections to work properly we first have to setup some things.

### Create resource

If you don't create an Azure OpenAI resource yet, you need to create it by following steps.  

1. Go to https://portal.azure.com/#create/Microsoft.CognitiveServicesOpenAI
1. Fill out all mandatory parameters. then create resource.
1. Go to resource page that has been created.

### Get keys and endpoint name.

You have to get the access token and endpoint name to call the API.

1. Go to resource page that has been created.
1. Click on [Keys and Endpoint]
1. Find your API key and Endpoint name.
1. Set these to the variables for using by script.

Note: Two keys are provided as standard for rotation, but only one of them is needed.

![image](./images/azure_keys_and_endpoint_01.png)

In [ ]:
# imports
Import-Module ..\PSOpenAI.psd1

$AuthType = 'azure'
$global:OPENAI_API_KEY = '<Put your api key here>'
$global:OPENAI_API_BASE  = 'https://<resource-name>.openai.azure.com/'

### (Optional) Use Azure Active Directory Authentication

You can get a user-based token from Azure AD by logging in with the [Az.Accounts](https://www.powershellgallery.com/packages/Az.Accounts/) PowerShell module or Azure CLI tools. This way you are secured by MFA and no need for a API Key.

Users logging in with Azure AD must be assigned a `Cognitive Services User` role or higher privileges.

Roles can be assigned from the [Access Control (IAM)] in the resource page.

![image](./images/azure_iam_01.png)

In [ ]:
# To run the following code, you need to install Az.Accounts PowerShell module.
# Install-Module Az.Accounts
Import-Module Az.Accounts

# Login with Azure AD
Connect-AzAccount

# Retrive access token
$MyToken = Get-AzAccessToken -ResourceUrl 'https://cognitiveservices.azure.com'

# Set to variables
$AuthType = 'azure_ad'  # You need to set AuthType as "azure_ad".
$global:OPENAI_API_KEY = $MyToken.Token


### Create model deployments manually

In Azure, the AI model to be used must be deployed under an arbitrary name.

1. Go to [Azure OpenAI Studio](https://oai.azure.com/) page.
1. Click on the [Deployments]
1. Click [Create new deployment], give it a name, select a model and version, then click [Create].

![image](./images/azure_model_deployments_01.png)


In [ ]:
$DeploymentName = '<Put your deployment name here>'

### (Optional) Create model deployments programatically

PSOpenAI has helper functions to manage model deployments.

Note 1 : You can only one deployment per model.  
Note 2 : Currently, you can not specify the name of deployments from script. The name will be set automatically by Azure API.


In [ ]:
# Now let's create the deployment
$model = 'gpt-35-turbo'
echo "Creating a new deployment with model: $model"
$deploy = New-AzureOpenAIDeployments -Model $model -ScaleType 'standard'
$DeploymentName = $deploy.id
echo "Successfully created deployment with id: $($deploy.id)"
echo $deploy

Creating a new deployment with model: gpt-35-turbo


Successfully created deployment with id: deployment-e59fa1085d07498983986b06bf6b5629

scale_settings : @{scale_type=standard}
model          : gpt-35-turbo
owner          : organization-owner
id             : deployment-e59fa1085d07498983986b06bf6b5629
status         : succeeded
object         : deployment
created_at     : 2023/04/29 18:25:26
updated_at     : 2023/04/29 18:25:26




## Create chat completion

Now let's send a sample chat completion to the deployment.

In [ ]:
# Need to set these variables properly in the above codes.
# $AuthType
# $DeploymentName
# $global:OPENAI_API_KEY
# $global:OPENAI_API_BASE

Request-AzureChatCompletion `
  -Message 'Hello Azure OpenAI Service.' `
  -Deployment $DeploymentName `
  -AuthType $AuthType


id      : chatcmpl-7Ab7oNOpy1dKm5bvhld6oRy6dlpXi
object  : chat.completion
model   : gpt-35-turbo
choices : {@{index=0; finish_reason=stop; message=}}
usage   : @{completion_tokens=9; prompt_tokens=14; total_tokens=23}
created : 2023/04/29 18:25:44
Message : Hello Azure OpenAI Service.
Answer  : {Hello! How may I assist you today?}
History : {System.Collections.Specialized.OrderedDictionary, System.Collections.Specialized.Ordered
          Dictionary}




### (Optional) Delete deployments programatically

Finally, let's delete the deployment.

In [ ]:
echo "Deleting deployment: $DeploymentName"
Remove-AzureOpenAIDeployments -Deployment $DeploymentName -Force

Deleting deployment: deployment-e59fa1085d07498983986b06bf6b5629
